In [1]:
%matplotlib inline

import sys

sys.path.append("..")

# Image classification

I will write a simple classifier for sketchy images to use as a benchmark for other methods.

In [5]:
# Load the Dataset class

import os

from src.datasets.sketchy import SketchyImages

dataset = SketchyImages()

In [7]:
import torch

# Number of workers for dataloader
workers = 4

# Batch size during training
batch_size = 128

# Number of GPUs available. Use 0 for CPU mode.
ngpu = 1

# Decide which device we want to run on
device = torch.device("cuda:0" if (torch.cuda.is_available() and ngpu > 0) else "cpu")